# Format State Parks

This notebook picks up after `format_national_parks.ipynb` to format the state park data.

In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import xarray as xr
import pandas as pd
import os

data_dir = "/pool0/home/steinadi/data/drought/DatabaseDrafts"

Populating the interactive namespace from numpy and matplotlib


### Explore Data

Path notes from `explore.ipynb`

In [2]:
# Good to go
# ----------
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Overnight", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Total", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Day Use", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Day User Monthly", skiprows=np.arange(1,4), index_col=[0,1,2], header=1)
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Overnight Monthly", skiprows=np.arange(1,4), index_col=[0,1,2], header=1)

# Needs splicing
# --------------
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Annual Day Use", skiprows=np.arange(1,6), index_col=0, header=1) # some totals that I think can be cut out?
# pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Annual Overnight", skiprows=np.arange(1,7), index_col=2, header=1) # also some strange totals

Read in data

In [3]:
# uses year-park multi-index with months for columns
wa_overnight = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Overnight", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)
wa_total = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Total", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)
wa_day_use = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="WA Day Use", skiprows=np.arange(1,5), usecols='B:Q', index_col=[0,1,2], header=1)

# uses park-month_num-year for index with sums for column
or_day_user_monthly = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Day User Monthly", skiprows=np.arange(1,4), index_col=[0,1,2], header=1)
or_overnight_monthly = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Overnight Monthly", skiprows=np.arange(1,4), index_col=[0,1,2], header=1)

# uses park/totals for index, year for columns
or_annual_day = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Annual Day Use", skiprows=np.arange(1,6), index_col=0, header=1)
or_annual_overnight = pd.read_excel(f"{data_dir}/StatePark_Visitations.xlsx", sheet_name="OR Annual Overnight", skiprows=np.arange(1,7), index_col=2, header=1)

Alright, in `format_national_parks.ipynb`, I got the netcdf to have dimensions of time and park, so I want to repeat that here I think.

First I want to take care of the splicing needs of the last two dataframes

After second thought, I want to keep those totals, could be useful, might as well not throw out data.

let's get a list of park names

In [4]:
wa_parks = []

for data in [wa_overnight, wa_total, wa_day_use]:
    for dex in data.index:
        park = dex[2].upper()
        if park not in wa_parks:
            wa_parks.append(park)

In [5]:
or_parks = []

for data in [or_day_user_monthly, or_overnight_monthly]:
    for dex in data.index:
        park = dex[0].upper()
        if park not in or_parks:
            or_parks.append(park)

for data in [or_annual_day, or_annual_overnight]:
    for dex in data.index:
        if isinstance(dex, str):
            park = dex.upper()
            if park not in or_parks:
                or_parks.append(park)

Got that. Now let's check our times. 

WA data goes from 2014 to 2020    
OR data goes from 1989 to 2021 for monthly, and 2002 to 2020 for annual

Hmmm ... let's see whether the annual and monthly data adds up for OR, (because if they do, then I might as well only store the monthly data and just compute the annual data later to make my coordinates easier in the netcdf)

In [6]:
or_day_user_monthly.head()

SumOfattendance
property                          recordedMonth recordedYear                 
Agate Beach State Recreation Site 7             1989                  37656.0
                                  8             1989                  35544.0
                                  9             1989                  28316.0
                                  10            1989                  16580.0
                                  11            1989                  11792.0

In [7]:
or_reproduced_day_annual = or_day_user_monthly.groupby(['recordedYear', 'property']).sum()
or_reproduced_day_annual.loc[2020]

,SumOfattendance
property,
Agate Beach State Recreation Site,158136.0
Alderwood State Wayside,51058.0
Alfred A. Loeb State Park,122526.0
Arcadia Beach State Recreation Site,126172.0
Arizona Beach,43130.0
...,...
Winchuck State Recreation Site,12264.0
Wyeth State Recreation Area,145290.0
Yachats Ocean Road State Natural Site,324644.0


In [11]:
or_annual_day.iloc[:,-1]

Location
AGATE BEACH STATE RECREATION SITE           158136.0
ALFRED A LOEB STATE PARK                    122526.0
ALSEA BAY HISTORIC INTERPRETIVE CENTER           NaN
ARCADIA BEACH STATE RECREATION SITE         126172.0
ARIZONA BEACH                                43130.0
                                             ...    
WARM SPRINGS STATE RECREATION SITE          215644.0
WHITE RIVER FALLS STATE PARK                 45044.0
Total Mountain Region                      9833322.5
NaN                                              NaN
Total Statewide                           42912867.5
Name: 2020 (updated 2-10-2021), Length: 186, dtype: float64

Alright, checked a few by eye (I know not very rigorous but didn't want to spend a bunch of time on this) and they are matching up, so I'll just keep the monthly totals and make life easier for me.

So let's make 2 datasets, one for WA parks and one for OR parks, just to keep things organized since they are state parks anywho. I'll have them as both monthly intervals for their respective time periods.

### WA State Parks

In [22]:
months = list(wa_day_use.columns)[:-1]
print(months)

['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']


In [29]:
wa_years = list(set([dex[0] for dex in wa_day_use.index]))
wa_years

[2016, 2017, 2018, 2019, 2020, 2014, 2015]

In [ ]:
revised_wa_park_visits = pd.DataFrame(index=pd.date_range(start=f"{wa_years[0]}", end=f"{wa_years[-1]}", freq="MS"), columns=wa_parks)

errors = []

for park in wa_parks:
    for year in wa_years:
        for month_num, month in enumerate(months):
            try:
                revised_wa_park_visits[]